In [71]:
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
import warnings
import numpy
warnings.filterwarnings('ignore')

from plotnine import *

In [72]:
read_file = pd.read_excel ("MSN Codes.xlsx")
read_file.to_csv ("MSN Codes.csv", index = None, header=True)
read_file = pd.read_excel ("Investment_Data_Testing.xlsx")
read_file.to_csv ("Investment_Data_Testing.csv", index = None, header=True)
# read_file = pd.read_excel ("Investment_Data_2020.xlsx")
# read_file.to_csv ("Investment_Data_2020.csv", index = None, header=True)

In [73]:
msn_code = pd.read_csv('MSN Codes.csv')

investment_data = pd.read_csv("Investment_Data_Testing.csv")

In [74]:
single_variables = investment_data[[
    'StateCode', 'Year', 'State',
    'CO2 Emissions (Mmt)', 'TotalNumberofInvestments', 'TotalAmountofAssistance'
]].drop_duplicates()

multi_variables = investment_data[[
    'StateCode', 'Year', 'State',
    'MSN', 'Amount'
]].drop_duplicates()


MSN_feature_pivot = multi_variables.pivot(index=['StateCode', 'Year', 'State'], columns='MSN', values='Amount')
# MSN_feature_pivot[MSN_feature_pivot.isna().any(axis=1)]


# tidy_data = MSN_feature_pivot.merge(single_variables, left_on=['StateCode', 'Year'], right_on=['StateCode', 'Year'])
# past_assistance = numpy.array([0])
# past_assistance = numpy.append(past_assistance, tidy_data.iloc[:, -1])
# past_assistance = numpy.delete(past_assistance, -1)
# tidy_data['PastAssistance'] = past_assistance
tidy_data = MSN_feature_pivot.merge(single_variables, left_on=['StateCode', 'Year'], right_on=['StateCode', 'Year'])
tidy_data_wo_na_response = tidy_data[~tidy_data['TotalAmountofAssistance'].isna()].drop(columns=['State'])
#df_filtered = tidy_data_wo_na_response[tidy_data_wo_na_response['Year'] > 2015]

# corr_tidy = df_filtered.corr(method = 'spearman', numeric_only=True)
# corr_last3 = corr_tidy.iloc[:, -4:]
# corr_last3.loc[corr_last3['TotalAmountofAssistance'] > 0.4]

In [75]:
tidy_data_wo_na_response
pastassistance = tidy_data_wo_na_response[['StateCode', 'Year', 'TotalAmountofAssistance']]
pastassistance['Year'] += 1
pastassistance.rename(columns={'TotalAmountofAssistance': 'pastAssistance'}, inplace=True)
tidy_data_with_pastassistance = tidy_data_wo_na_response.merge(pastassistance,  left_on=['StateCode', 'Year'], right_on=['StateCode', 'Year'])

tidy_input_data = tidy_data_with_pastassistance.drop(columns=['TotalAmountofAssistance'])
tidy_response_data = tidy_data_with_pastassistance[['StateCode', 'Year', 'TotalAmountofAssistance']]

tidy_input_data



,StateCode,Year,BDFDB,BDPRP,BFFDB,BFPRP,CLPRB,CLPRK,CLPRP,COPRK,...,TETCB,WDEXB,WDPRB,WDTCB,WSTCB,WWPRB,WYTCB,CO2 Emissions (Mmt),TotalNumberofInvestments,pastAssistance
0,AK,2016,27.0,5.0,27.0,5.0,13942.0,14.957,932.0,5.722,...,593662.0,0.0,7636.0,7636.0,412.0,8048.0,1563.0,33.405611,14.0,3345612.0
1,AK,2017,29.0,5.0,29.0,5.0,14365.0,14.978,959.0,5.723,...,601492.0,0.0,6501.0,6501.0,436.0,6937.0,1305.0,33.729130,18.0,2247734.0
2,AK,2018,15.0,3.0,15.0,3.0,13752.0,15.253,902.0,5.706,...,604644.0,0.0,6910.0,6910.0,456.0,7366.0,1411.0,34.514589,44.0,13509662.0
3,AK,2019,0.0,0.0,0.0,0.0,14867.0,15.252,975.0,5.698,...,605689.0,0.0,6418.0,6418.0,386.0,6803.0,1270.0,34.259440,32.0,18236626.0
4,AK,2020,0.0,0.0,0.0,0.0,15523.0,15.206,1021.0,5.691,...,640132.0,0.0,7423.0,7423.0,407.0,7831.0,1128.0,35.967527,37.0,13734021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,WY,2016,0.0,0.0,0.0,0.0,5169939.0,17.394,297218.0,5.722,...,504984.0,0.0,4357.0,4357.0,2.0,4358.0,40522.0,61.259993,23.0,249960.0
246,WY,2017,0.0,0.0,0.0,0.0,5516757.0,17.433,316454.0,5.723,...,534840.0,0.0,5049.0,5049.0,0.0,5049.0,39806.0,62.796487,47.0,1359346.0
247,WY,2018,0.0,0.0,0.0,0.0,5315953.0,17.476,304188.0,5.706,...,557604.0,0.0,4934.0,4934.0,0.0,4934.0,36936.0,63.846081,53.0,413987.0
248,WY,2019,0.0,0.0,0.0,0.0,4828513.0,17.437,276912.0,5.698,...,546671.0,0.0,4969.0,4969.0,0.0,4969.0,37070.0,59.096649,32.0,573639.0


In [76]:
filtered_MSN_lst = ['BFPRP', 'CLPRB', 'ENPRP', 'GETCB', 'HYTCB', 'NGMPB', 'NUETB', 'PAPRB', 'REPRB', 'SOTCB', 'TETCB', 'WWPRB', 'WYTCB']
filtered_MSN_lst = ['BDFDB', 'BDPRP', 'BFFDB', 'BFPRP', 'CLPRB',
       'CLPRK', 'CLPRP', 'COPRK', 'EMFDB', 'ENPRP', 'GETCB', 'HYTCB', 'NCPRB',
       'NGMPB', 'NGMPK', 'NGMPP', 'NUETB', 'PAPRB', 'PAPRP', 'REPRB', 'SOTCB',
       'TEPRB', 'TETCB', 'WDEXB', 'WDPRB', 'WDTCB', 'WSTCB', 'WWPRB', 'WYTCB']

corr_tidy = tidy_data_wo_na_response.corr(method = 'spearman', numeric_only=True)
corr_last3 = corr_tidy.iloc[:, -3:]
corr_last3.loc[(corr_last3['TotalAmountofAssistance'] > 0.1) ]
MSN_filtered_df = corr_tidy.iloc[:, -3:].reset_index()
MSN_filtered_df[MSN_filtered_df['index'].isin(filtered_MSN_lst)]

,index,CO2 Emissions (Mmt),TotalNumberofInvestments,TotalAmountofAssistance
1,BDFDB,0.448764,0.379818,0.281415
2,BDPRP,0.453327,0.375464,0.284464
3,BFFDB,0.411717,0.414871,0.293806
4,BFPRP,0.414695,0.411883,0.296219
5,CLPRB,0.320402,-0.013039,-0.172242
6,CLPRK,0.349326,0.003760,-0.133785
7,CLPRP,0.314836,-0.012886,-0.167603
8,COPRK,0.323508,0.152856,-0.071485
9,EMFDB,0.380828,0.364143,0.260857
10,ENPRP,0.380817,0.364047,0.260809


In [77]:
filtered_columns = ['StateCode', 'Year'] + filtered_MSN_lst + ['CO2 Emissions (Mmt)', 'TotalNumberofInvestments', 'pastAssistance']

filtered_input_data = tidy_input_data[filtered_columns]
stateCodes = filtered_input_data.StateCode.unique()
filtered_input_data

,StateCode,Year,BDFDB,BDPRP,BFFDB,BFPRP,CLPRB,CLPRK,CLPRP,COPRK,...,TETCB,WDEXB,WDPRB,WDTCB,WSTCB,WWPRB,WYTCB,CO2 Emissions (Mmt),TotalNumberofInvestments,pastAssistance
0,AK,2016,27.0,5.0,27.0,5.0,13942.0,14.957,932.0,5.722,...,593662.0,0.0,7636.0,7636.0,412.0,8048.0,1563.0,33.405611,14.0,3345612.0
1,AK,2017,29.0,5.0,29.0,5.0,14365.0,14.978,959.0,5.723,...,601492.0,0.0,6501.0,6501.0,436.0,6937.0,1305.0,33.729130,18.0,2247734.0
2,AK,2018,15.0,3.0,15.0,3.0,13752.0,15.253,902.0,5.706,...,604644.0,0.0,6910.0,6910.0,456.0,7366.0,1411.0,34.514589,44.0,13509662.0
3,AK,2019,0.0,0.0,0.0,0.0,14867.0,15.252,975.0,5.698,...,605689.0,0.0,6418.0,6418.0,386.0,6803.0,1270.0,34.259440,32.0,18236626.0
4,AK,2020,0.0,0.0,0.0,0.0,15523.0,15.206,1021.0,5.691,...,640132.0,0.0,7423.0,7423.0,407.0,7831.0,1128.0,35.967527,37.0,13734021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,WY,2016,0.0,0.0,0.0,0.0,5169939.0,17.394,297218.0,5.722,...,504984.0,0.0,4357.0,4357.0,2.0,4358.0,40522.0,61.259993,23.0,249960.0
246,WY,2017,0.0,0.0,0.0,0.0,5516757.0,17.433,316454.0,5.723,...,534840.0,0.0,5049.0,5049.0,0.0,5049.0,39806.0,62.796487,47.0,1359346.0
247,WY,2018,0.0,0.0,0.0,0.0,5315953.0,17.476,304188.0,5.706,...,557604.0,0.0,4934.0,4934.0,0.0,4934.0,36936.0,63.846081,53.0,413987.0
248,WY,2019,0.0,0.0,0.0,0.0,4828513.0,17.437,276912.0,5.698,...,546671.0,0.0,4969.0,4969.0,0.0,4969.0,37070.0,59.096649,32.0,573639.0


In [78]:
X_variables = filtered_input_data#.drop(columns=['StateCode', 'Year'])
y_variables = tidy_response_data#.drop(columns=['StateCode', 'Year'])

train_X = X_variables[X_variables.Year < 2020]
train_y = y_variables[y_variables.Year < 2020]
test_X = X_variables[X_variables.Year == 2020]
compare_results = y_variables[y_variables.Year == 2020]

In [79]:
"""
Separated by state
"""

compare_results_total = None

for state in stateCodes:
    state_train_X = train_X[train_X['StateCode']==state].drop(columns=['StateCode', 'Year'])
    state_train_y = train_y[train_y['StateCode']==state].drop(columns=['StateCode', 'Year'])
    state_test_X = test_X[test_X['StateCode']==state].drop(columns=['StateCode', 'Year'])
    
#     poly = PolynomialFeatures(2, interaction_only=False)
#     poly.fit(state_train_X)
#     poly_train_X = poly.transform(state_train_X)
#     poly_test_X = poly.transform(state_test_X)

    poly_train_X = state_train_X
    poly_test_X = state_test_X
    
    scaler = StandardScaler()
    scaler.fit(poly_train_X)
    poly_train_X = scaler.transform(poly_train_X)
    poly_test_X = scaler.transform(poly_test_X)
    
    regr = ElasticNet(random_state=0, l1_ratio=0.3, alpha = 10)
    regr.fit(poly_train_X, state_train_y)
    predictions = regr.predict(poly_test_X)
    
    compare_results_poly = compare_results[compare_results['StateCode']==state]
    compare_results_poly['predict'] = predictions.tolist()
    compare_results_poly['ans_div_predict'] = compare_results_poly['TotalAmountofAssistance'] / compare_results_poly['predict']
    
    if compare_results_total is None:
        compare_results_total = compare_results_poly
    else:
        compare_results_total = pd.concat([compare_results_total, compare_results_poly])
    
print('RSME: ', ((compare_results_total.predict - compare_results_total.TotalAmountofAssistance) ** 2).mean() ** .5)
compare_results_total


RSME:  168238304.2208757


,StateCode,Year,TotalAmountofAssistance,predict,ans_div_predict
4,AK,2020,9567164.0,1.715850e+07,0.557576
9,AL,2020,1174323.0,1.116203e+06,1.052069
14,AR,2020,61208016.0,2.374662e+07,2.577546
19,AZ,2020,23756565.0,4.134964e+08,0.057453
24,CA,2020,58722209.0,2.593581e+08,0.226414
29,CO,2020,6304206.0,1.091925e+07,0.577348
34,CT,2020,491729.0,1.062570e+06,0.462773
39,DE,2020,23750266.0,1.662507e+06,14.285812
44,FL,2020,8467648.0,6.998228e+07,0.120997
49,GA,2020,7307409.0,-6.757085e+06,-1.081444


In [80]:
train_X = train_X.drop(columns=['StateCode', 'Year'])
train_y = train_y.drop(columns=['StateCode', 'Year'])
test_X = test_X.drop(columns=['StateCode', 'Year'])

scaler = StandardScaler()
scaler.fit(train_X)
train_X = scaler.transform(train_X)
test_X = scaler.transform(test_X)


regr = ElasticNet(random_state=0, l1_ratio=0.5, alpha = 1)
regr.fit(train_X, train_y)
predictions = regr.predict(test_X)

compare_results['predict'] = predictions.tolist()
compare_results['ans_div_predict'] = compare_results['TotalAmountofAssistance'] / compare_results['predict']

print('RSME: ', ((compare_results.predict - compare_results.TotalAmountofAssistance) ** 2).mean() ** .5)

from sklearn.metrics import r2_score
print(r2_score(compare_results['TotalAmountofAssistance'], predictions))
compare_results

RSME:  27880051.996203378
0.3220347473398373


,StateCode,Year,TotalAmountofAssistance,predict,ans_div_predict
4,AK,2020,9567164.0,7.821391e+06,1.223205
9,AL,2020,1174323.0,1.510746e+07,0.077731
14,AR,2020,61208016.0,2.236465e+07,2.736819
19,AZ,2020,23756565.0,3.658632e+07,0.649329
24,CA,2020,58722209.0,9.250308e+07,0.634814
29,CO,2020,6304206.0,1.206081e+07,0.522702
34,CT,2020,491729.0,1.021697e+07,0.048129
39,DE,2020,23750266.0,1.052246e+07,2.257102
44,FL,2020,8467648.0,4.768639e+07,0.177570
49,GA,2020,7307409.0,5.450338e+07,0.134073


In [89]:
"""
Andrew original code
"""
X_variables = tidy_input_data#.drop(columns=['StateCode', 'Year'])
y_variables = tidy_response_data#.drop(columns=['StateCode', 'Year'])


train_X = X_variables[(X_variables.Year < 2020) & (X_variables.Year > 2015)].drop(columns=['StateCode', 'Year'])
#print(train_X)
train_y = y_variables[(y_variables.Year < 2020) & (y_variables.Year > 2015)].drop(columns=['StateCode', 'Year'])
test_X = X_variables[X_variables.Year == 2020].drop(columns=['StateCode', 'Year'])
compare_results = y_variables[y_variables.Year == 2020]

scaler = StandardScaler()
scaler.fit(train_X)
train_X = scaler.transform(train_X)
test_X = scaler.transform(test_X)


regr = ElasticNet(random_state=0, l1_ratio=0.5, alpha = 1)
regr.fit(train_X, train_y)
predictions = regr.predict(test_X)

compare_results['predict'] = predictions.tolist()
compare_results['ans_div_predict'] = compare_results['TotalAmountofAssistance'] / compare_results['predict']

print('RSME: ', ((compare_results.predict - compare_results.TotalAmountofAssistance) ** 2).mean() ** .5)

from sklearn.metrics import r2_score
print(r2_score(compare_results['TotalAmountofAssistance'], predictions))
compare_results
tidy_input_data

RSME:  27880051.996203378
0.3220347473398373


,StateCode,Year,BDFDB,BDPRP,BFFDB,BFPRP,CLPRB,CLPRK,CLPRP,COPRK,...,TETCB,WDEXB,WDPRB,WDTCB,WSTCB,WWPRB,WYTCB,CO2 Emissions (Mmt),TotalNumberofInvestments,pastAssistance
0,AK,2016,27.0,5.0,27.0,5.0,13942.0,14.957,932.0,5.722,...,593662.0,0.0,7636.0,7636.0,412.0,8048.0,1563.0,33.405611,14.0,3345612.0
1,AK,2017,29.0,5.0,29.0,5.0,14365.0,14.978,959.0,5.723,...,601492.0,0.0,6501.0,6501.0,436.0,6937.0,1305.0,33.729130,18.0,2247734.0
2,AK,2018,15.0,3.0,15.0,3.0,13752.0,15.253,902.0,5.706,...,604644.0,0.0,6910.0,6910.0,456.0,7366.0,1411.0,34.514589,44.0,13509662.0
3,AK,2019,0.0,0.0,0.0,0.0,14867.0,15.252,975.0,5.698,...,605689.0,0.0,6418.0,6418.0,386.0,6803.0,1270.0,34.259440,32.0,18236626.0
4,AK,2020,0.0,0.0,0.0,0.0,15523.0,15.206,1021.0,5.691,...,640132.0,0.0,7423.0,7423.0,407.0,7831.0,1128.0,35.967527,37.0,13734021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,WY,2016,0.0,0.0,0.0,0.0,5169939.0,17.394,297218.0,5.722,...,504984.0,0.0,4357.0,4357.0,2.0,4358.0,40522.0,61.259993,23.0,249960.0
246,WY,2017,0.0,0.0,0.0,0.0,5516757.0,17.433,316454.0,5.723,...,534840.0,0.0,5049.0,5049.0,0.0,5049.0,39806.0,62.796487,47.0,1359346.0
247,WY,2018,0.0,0.0,0.0,0.0,5315953.0,17.476,304188.0,5.706,...,557604.0,0.0,4934.0,4934.0,0.0,4934.0,36936.0,63.846081,53.0,413987.0
248,WY,2019,0.0,0.0,0.0,0.0,4828513.0,17.437,276912.0,5.698,...,546671.0,0.0,4969.0,4969.0,0.0,4969.0,37070.0,59.096649,32.0,573639.0


In [88]:
X_variables

,StateCode,Year,BDFDB,BDPRP,BFFDB,BFPRP,CLPRB,CLPRK,CLPRP,COPRK,...,TETCB,WDEXB,WDPRB,WDTCB,WSTCB,WWPRB,WYTCB,CO2 Emissions (Mmt),TotalNumberofInvestments,pastAssistance
0,AK,2016,27.0,5.0,27.0,5.0,13942.0,14.957,932.0,5.722,...,593662.0,0.0,7636.0,7636.0,412.0,8048.0,1563.0,33.405611,14.0,3345612.0
1,AK,2017,29.0,5.0,29.0,5.0,14365.0,14.978,959.0,5.723,...,601492.0,0.0,6501.0,6501.0,436.0,6937.0,1305.0,33.729130,18.0,2247734.0
2,AK,2018,15.0,3.0,15.0,3.0,13752.0,15.253,902.0,5.706,...,604644.0,0.0,6910.0,6910.0,456.0,7366.0,1411.0,34.514589,44.0,13509662.0
3,AK,2019,0.0,0.0,0.0,0.0,14867.0,15.252,975.0,5.698,...,605689.0,0.0,6418.0,6418.0,386.0,6803.0,1270.0,34.259440,32.0,18236626.0
4,AK,2020,0.0,0.0,0.0,0.0,15523.0,15.206,1021.0,5.691,...,640132.0,0.0,7423.0,7423.0,407.0,7831.0,1128.0,35.967527,37.0,13734021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,WY,2016,0.0,0.0,0.0,0.0,5169939.0,17.394,297218.0,5.722,...,504984.0,0.0,4357.0,4357.0,2.0,4358.0,40522.0,61.259993,23.0,249960.0
246,WY,2017,0.0,0.0,0.0,0.0,5516757.0,17.433,316454.0,5.723,...,534840.0,0.0,5049.0,5049.0,0.0,5049.0,39806.0,62.796487,47.0,1359346.0
247,WY,2018,0.0,0.0,0.0,0.0,5315953.0,17.476,304188.0,5.706,...,557604.0,0.0,4934.0,4934.0,0.0,4934.0,36936.0,63.846081,53.0,413987.0
248,WY,2019,0.0,0.0,0.0,0.0,4828513.0,17.437,276912.0,5.698,...,546671.0,0.0,4969.0,4969.0,0.0,4969.0,37070.0,59.096649,32.0,573639.0
